In [1]:
import cv2
import os
import numpy as np

In [6]:
# Read and resize images
import cv2
import os

# Specify the target size (e.g., 224x224 pixels)
target_size = (224, 224)

# Define the directory containing your images
data_dir = 'data/catsvsdogs/train/cats'

# Loop through each image in the directory and resize it
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):  # Assuming your images are in JPEG format
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, target_size)
        cv2.imwrite(img_path, img)

In [7]:
# Do the same for rest of the data

In [8]:
target_size = (224, 224)

# Define the directory containing your images
data_dir = 'data/catsvsdogs/train/dogs'

# Loop through each image in the directory and resize it
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):  # Assuming your images are in JPEG format
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, target_size)
        cv2.imwrite(img_path, img)

In [9]:
target_size = (224, 224)

# Define the directory containing your images
data_dir = 'data/catsvsdogs/test/dogs'

# Loop through each image in the directory and resize it
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):  # Assuming your images are in JPEG format
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, target_size)
        cv2.imwrite(img_path, img)

In [10]:
target_size = (224, 224)

# Define the directory containing your images
data_dir = 'data/catsvsdogs/test/cats'

# Loop through each image in the directory and resize it
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):  # Assuming your images are in JPEG format
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, target_size)
        cv2.imwrite(img_path, img)

In [2]:
import cv2
import os

# Define the directory containing your images
data_dir = 'data/catsvsdogs/train/cats'

# Initialize lists to store image data and corresponding labels (if available)
train_cats_imgs = []
train_dogs_imgs = []
test_cats_imgs = []
test_dogs_imgs = []

# Loop through each image in the directory
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):  # Assuming your images are in JPEG format
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        # Optionally, you can resize the image here if needed
        # img = cv2.resize(img, (width, height))
        train_cats_imgs.append(img)
        # If you have a label associated with the image, add it to the labels list
        # labels.append(label)

data_dir = 'data/catsvsdogs/train/dogs'
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):  # Assuming your images are in JPEG format
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        train_dogs_imgs.append(img)

data_dir = 'data/catsvsdogs/test/dogs'
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):  # Assuming your images are in JPEG format
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        test_dogs_imgs.append(img)
        

data_dir = 'data/catsvsdogs/test/cats'
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):  # Assuming your images are in JPEG format
        img_path = os.path.join(data_dir, filename)
        img = cv2.imread(img_path)
        test_cats_imgs.append(img)

# Convert the lists to NumPy arrays if needed
train_cats_imgs = np.array(train_cats_imgs)
train_dogs_imgs = np.array(train_dogs_imgs)
test_cats_imgs = np.array(test_cats_imgs)
test_dogs_imgs = np.array(test_dogs_imgs)

In [3]:
# Z-score normalization of the data
# might be better if we simply normalized them into a range like 0-1 with a scaler
# should work for now

In [4]:
# Calculate the mean and standard deviation for the training data (both cats and dogs combined)
mean_training = np.mean(np.concatenate((train_cats_imgs, train_dogs_imgs), axis=0), axis=(0, 1, 2))
std_training = np.std(np.concatenate((train_cats_imgs, train_dogs_imgs), axis=0), axis=(0, 1, 2))

# Normalize both training and test data using the statistics from the training data
train_cats_normalized = (train_cats_imgs - mean_training) / std_training
train_dogs_normalized = (train_dogs_imgs - mean_training) / std_training

test_cats_normalized = (test_cats_imgs - mean_training) / std_training
test_dogs_normalized = (test_dogs_imgs - mean_training) / std_training

In [5]:
# import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [6]:
# Creating the model

model = Sequential()

# Convolutional layers with L2 regularization
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D((2, 2)))

# Flatten the output from convolutional layers
model.add(Flatten())

# Fully connected layers with L2 regularization
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))  # Output layer for binary classification

In [7]:
# Training the model
# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile the model as before
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with early stopping
history = model.fit(np.concatenate((train_cats_normalized, train_dogs_normalized), axis=0),
                    np.concatenate((np.zeros(len(train_cats_normalized)), np.ones(len(train_dogs_normalized)))),
                    validation_data=(np.concatenate((test_cats_normalized, test_dogs_normalized), axis=0),
                                    np.concatenate((np.zeros(len(test_cats_normalized)), np.ones(len(test_dogs_normalized))))),
                    epochs=100,  # Adjust as needed
                    batch_size=32,  # Adjust as needed
                    callbacks=[early_stopping])  # Use early stopping

Epoch 1/100
18/18 [==============================] - 12s 647ms/step - loss: 4.3382 - accuracy: 0.5081 - val_loss: 3.8960 - val_accuracy: 0.5286
Epoch 2/100
18/18 [==============================] - 12s 644ms/step - loss: 3.5990 - accuracy: 0.5835 - val_loss: 3.3388 - val_accuracy: 0.5286
Epoch 3/100
18/18 [==============================] - 12s 643ms/step - loss: 3.0973 - accuracy: 0.6176 - val_loss: 2.9430 - val_accuracy: 0.5500
Epoch 4/100
18/18 [==============================] - 12s 669ms/step - loss: 2.7334 - accuracy: 0.6732 - val_loss: 2.6753 - val_accuracy: 0.5643
Epoch 5/100
18/18 [==============================] - 13s 700ms/step - loss: 2.4721 - accuracy: 0.7199 - val_loss: 2.4804 - val_accuracy: 0.5929
Epoch 6/100
18/18 [==============================] - 12s 685ms/step - loss: 2.3082 - accuracy: 0.7181 - val_loss: 2.3399 - val_accuracy: 0.6071
Epoch 7/100
18/18 [==============================] - 12s 644ms/step - loss: 2.1706 - accuracy: 0.7002 - val_loss: 2.2202 - val_accuracy:

In [8]:
test_loss, test_accuracy = model.evaluate(np.concatenate((test_cats_normalized, test_dogs_normalized), axis=0),
                                          np.concatenate((np.zeros(len(test_cats_normalized)), np.ones(len(test_dogs_normalized)))))
print(f'Test Accuracy: {test_accuracy}')

5/5 [==============================] - 1s 106ms/step - loss: 0.6929 - accuracy: 0.5929
Test Accuracy: 0.5928571224212646
